In [ ]:
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukr_g2p"
export PYTHONPATH="${PYTHONPATH}:/data/exp/kostya/ukroASR"
UKROASR_ROOT=/data/exp/kostya/ukroASR
# export PYTHONIOENCODING=utf-8

In [ ]:
cd $UKROASR_ROOT

In [ ]:
# cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh

In [ ]:
train_cmd=run.pl

In [ ]:
work_dir=/data/exp/kostya/ukroASR

In [ ]:
data=$work_dir/data

In [ ]:
exp=$work_dir/exp

In [ ]:
cmd_prefix='cd $KALDI_ROOT/egs/wsj/s5; . ./path.sh; . ./cmd.sh;'

# TRAINING LANGUAGE MODEL

We are going to train LM without using dataset for training AM in order to not regenerate the whole model each time after adding small portion of data for AM training

### Order 3 LM for decoding with prunings

In [ ]:
dvc run \
-d $UKROASR_ROOT/artifacts/text_plane \
-d $UKROASR_ROOT/artifacts/normal_fiction.txt \
-d $UKROASR_ROOT/artifacts/normal_news.txt \
-d $UKROASR_ROOT/artifacts/normal_wiki_dump.txt \
-d $UKROASR_ROOT/artifacts/normal_ubercorpus.txt \
-o $data/lm_ukr_o3.gz \
-n lm_ukr_o3 \
"cut -d' ' -f2- $UKROASR_ROOT/artifacts/text_plane \
                $UKROASR_ROOT/artifacts/normal_fiction.txt \
                $UKROASR_ROOT/artifacts/normal_news.txt \
                $UKROASR_ROOT/artifacts/normal_wiki_dump.txt \
                $UKROASR_ROOT/artifacts/normal_ubercorpus.txt > /tmp/train_corpus; \
~/kenlm/build/bin/lmplz -o 3 --prune 6 8 10 </tmp/train_corpus | gzip -c > $data/lm_ukr_o3.gz"

In [ ]:
git add dvc.lock dvc.yaml data/.gitignore

### generating local/dict for lm training

In [ ]:
dvc run -f \
-d $UKROASR_ROOT/prepare_local_dict_from_lm.sh -d $data/lm_ukr_o3.gz -d $UKROASR_ROOT/artifacts/lexicon.txt \
-d $data/local/dict/nonsilence_phones.txt -d $data/local/dict/extra_questions.txt \
-d $data/local/dict/silence_phones.txt -d $data/local/dict/optional_silence.txt \
-o $data/local/dict_lm_train/vocab -o $data/local/dict_lm_train/lexicon.txt \
-n local_dict_lm_train \
$cmd_prefix $UKROASR_ROOT/prepare_local_dict_from_lm.sh --nj 16 $data/local/dict $data/lm_ukr_o3.gz $data/local/dict_lm_train

In [ ]:
git add dvc.yaml dvc.lock

### Order 4 LM for rescoring without prunings

In [ ]:
dvc run -f \
-d $UKROASR_ROOT/artifacts/text_plane \
-d $UKROASR_ROOT/artifacts/normal_fiction.txt \
-d $UKROASR_ROOT/artifacts/normal_news.txt \
-d $UKROASR_ROOT/artifacts/normal_wiki_dump.txt \
-d $UKROASR_ROOT/artifacts/normal_ubercorpus.txt \
-d $data/local/dict_lm_train/vocab \
-o $data/lm_ukr_o4.gz \
-n lm_ukr_o4 \
"cut -d' ' -f2- $UKROASR_ROOT/artifacts/text_plane \
                $UKROASR_ROOT/artifacts/normal_fiction.txt \
                $UKROASR_ROOT/artifacts/normal_news.txt \
                $UKROASR_ROOT/artifacts/normal_wiki_dump.txt \
                $UKROASR_ROOT/artifacts/normal_ubercorpus.txt > /tmp/train_corpus; \
~/kenlm/build/bin/lmplz -o 4 --limit_vocab_file $data/local/dict_lm_train/vocab </tmp/train_corpus | gzip -c > $data/lm_ukr_o4.gz"

In [ ]:
git add dvc.lock data/.gitignore dvc.yaml

### Generating Kaldi's lang dir

In [ ]:
dvc run -f \
-d $data/local/dict_lm_train/lexicon.txt \
-d $data/local/dict_lm_train/silence_phones.txt \
-d $data/local/dict_lm_train/nonsilence_phones.txt \
-d $data/local/dict_lm_train/optional_silence.txt \
-d $data/local/dict_lm_train/extra_questions.txt \
-o $data/lang/L.fst \
-o $data/lang/L_disambig.fst \
-o $data/lang/oov.int \
-o $data/lang/oov.txt \
-o $data/lang/phones.txt \
-o $data/lang/topo \
-o $data/lang/words.txt \
-o $data/lang/phones \
-O $data/local/dict_lm_train/lexiconp.txt \
-n lang_dir \
"$cmd_prefix utils/prepare_lang.sh --position-dependent-phones false $data/local/dict_lm_train '<UNK>' $data/local/lang_lm $data/lang"

In [ ]:
l $data/local/dict_lm_train

In [ ]:
git add dvc.yaml dvc.lock

### Formatting Kaldi's LM dir

In [ ]:
dvc run -f \
-d $data/lang/words.txt -d $data/lm_ukr_o3.gz -d $data/local/dict_lm_train/lexicon.txt \
-o $data/lang/G.fst \
-n lang_fst \
"$cmd_prefix utils/format_lm.sh $data/lang $data/lm_ukr_o3.gz $data/local/dict_lm_train/lexicon.txt $data/lang"

In [ ]:
git add dvc.yaml dvc.lock

### Generating rescore lm dir

In [ ]:
dvc run -f \
-d $data/lang/words.txt -d $data/lm_ukr_o4.gz \
-o $data/carpa/G.carpa \
-n carpa \
"$cmd_prefix utils/build_const_arpa_lm.sh $data/lm_ukr_o4.gz $data/lang $data/carpa"

In [ ]:
git add dvc.lock dvc.yaml

# IMPORT DATASET

In [ ]:
# import train.csv

In [ ]:
dvc import -o data/raw_data/train.csv \
https://github.com/kosti4ka/ukrodataset.git train.csv

In [ ]:
git add data/raw_data/train.csv.dvc

In [ ]:
# import dev.csv

In [ ]:
dvc import -o data/raw_data/dev.csv \
https://github.com/kosti4ka/ukrodataset.git dev.csv

In [ ]:
git add data/raw_data/dev.csv.dvc

In [ ]:
# import audio data itself

In [ ]:
dvc import -o data/raw_data \
https://github.com/kosti4ka/ukrodataset.git audiobooks

In [ ]:
git add -f data/raw_data/audiobooks.dvc

In [ ]:
dvc update data/raw_data/audiobooks.dvc

# GENERATING DATA FOLDERS

In [ ]:
# train data folder

In [ ]:
dvc run \
-d $data/raw_data/train.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/train/text -o $data/train/utt2spk -o $data/train/segments -o $data/train/spk2utt -o $data/train/wav.scp \
-n train_data_dir \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/train.csv $data/train

In [ ]:
# dev data folder

In [ ]:
dvc run \
-d $data/raw_data/dev.csv -d $UKROASR_ROOT/gen_data_dir.py \
-o $data/dev/text -o $data/dev/utt2spk -o $data/dev/segments -o $data/dev/spk2utt -o $data/dev/wav.scp \
-n dev_data_dir \
python3.6 $UKROASR_ROOT/gen_data_dir.py $data/raw_data/dev.csv $data/dev

In [ ]:
git add -f dvc.lock data/dev/.gitignore dvc.yaml

# GENERATING LANG DIR FOR AM TRAINING

In [ ]:
dvc run \
-d $UKROASR_ROOT/prepare_local_dict_from_text.sh -d $data/train/text -d $UKROASR_ROOT/artifacts/lexicon.txt \
-d $data/local/dict/nonsilence_phones.txt -d $data/local/dict/extra_questions.txt \
-d $data/local/dict/silence_phones.txt -d $data/local/dict/optional_silence.txt \
-o $data/local/dict_am_train/vocab -o $data/local/dict_am_train/lexicon.txt \
-n local_dict_am_train \
$cmd_prefix $UKROASR_ROOT/prepare_local_dict_from_text.sh --nj 16 $data/local/dict $data/local/dict_am_train $data/train/text

In [ ]:
git add dvc.yaml dvc.lock

In [ ]:
dvc run \
-d $data/local/dict_am_train/lexicon.txt \
-d $data/local/dict_am_train/silence_phones.txt \
-d $data/local/dict_am_train/nonsilence_phones.txt \
-d $data/local/dict_am_train/optional_silence.txt \
-d $data/local/dict_am_train/extra_questions.txt \
-o $data/lang_am_train/L.fst \
-o $data/lang_am_train/L_disambig.fst \
-o $data/lang_am_train/oov.int \
-o $data/lang_am_train/oov.txt \
-o $data/lang_am_train/phones.txt \
-o $data/lang_am_train/topo \
-o $data/lang_am_train/words.txt \
-o $data/lang_am_train/phones \
-O $data/local/dict_am_train/lexiconp.txt \
-n lang_dir_am_train \
"$cmd_prefix utils/prepare_lang.sh --position-dependent-phones false $data/local/dict_am_train '<UNK>' $data/local/lang_am $data/lang_am_train"

In [ ]:
git add dvc.yaml data/lang_am_train/.gitignore dvc.lock

# Computing features

In [ ]:
dvc run -f \
-d $data/train/wav.scp -d $data/train/segments -d $data/train/utt2spk \
-O $exp/train_mfcc -O $data/train/feats.scp \
-n mfcc_train \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/train $exp/log/mfcc $exp/train_mfcc"

In [ ]:
dvc run \
-d $data/dev/wav.scp -d $data/dev/segments -d $data/dev/utt2spk \
-O $exp/dev_mfcc -O $data/dev/feats.scp \
-n mfcc_dev \
"python3.6 $UKROASR_ROOT/features/make_mfcc.py $data/dev $exp/log/mfcc $exp/dev_mfcc"

In [ ]:
git add dvc.yaml dvc.lock

In [ ]:
# compute cmvn features

In [ ]:
for split in train dev
do
dvc run \
-d $data/$split/feats.scp -d $data/$split/utt2spk \
-O $exp/${split}_cmvn -O $data/$split/cmvn.scp \
-n cmvn_${split} \
"python3.6 $UKROASR_ROOT/features/compute_cmvn_stats.py $data/$split $exp/log/cmvn $exp/${split}_cmvn"
done

In [ ]:
git add dvc.yaml dvc.lock

# Sampling again

In [ ]:
dvc run \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_2kshort/text -o $data/train_2kshort/utt2spk -o $data/train_2kshort/segments -o $data/train_2kshort/spk2utt -o $data/train_2kshort/wav.scp -o $data/train_2kshort/feats.scp -o $data/train_2kshort/cmvn.scp \
-n train_2kshort_data \
"$cmd_prefix utils/subset_data_dir.sh --shortest $data/train 2000 $data/train_2kshort"

In [ ]:
dvc run \
-d $data/train/text -d $data/train/utt2spk -d $data/train/segments -d $data/train/spk2utt -d $data/train/wav.scp -d $data/train/feats.scp -d $data/train/cmvn.scp \
-o $data/train_5k/text -o $data/train_5k/utt2spk -o $data/train_5k/segments -o $data/train_5k/spk2utt -o $data/train_5k/wav.scp -o $data/train_5k/feats.scp -o $data/train_5k/cmvn.scp \
-n train_5k_data \
"$cmd_prefix utils/subset_data_dir.sh $data/train 5000 $data/train_5k"

In [ ]:
dvc run \
-d $data/dev/text -d $data/dev/utt2spk -d $data/dev/segments -d $data/dev/spk2utt -d $data/dev/wav.scp -d $data/dev/feats.scp -d $data/dev/cmvn.scp \
-o $data/dev_0.75kshort/text -o $data/dev_0.75kshort/utt2spk -o $data/dev_0.75kshort/segments -o $data/dev_0.75kshort/spk2utt -o $data/dev_0.75kshort/wav.scp -o $data/dev_0.75kshort/feats.scp -o $data/dev_0.75kshort/cmvn.scp \
-n dev_075kshort_data \
"$cmd_prefix utils/subset_data_dir.sh --shortest $data/dev 750 $data/dev_0.75kshort"

In [ ]:
dvc run \
-d $data/dev_0.75kshort/text -d $data/dev_0.75kshort/utt2spk -d $data/dev_0.75kshort/segments -d $data/dev_0.75kshort/spk2utt -d $data/dev_0.75kshort/wav.scp -d $data/dev_0.75kshort/feats.scp -d $data/dev_0.75kshort/cmvn.scp \
-o $data/dev_0.15k/text -o $data/dev_0.15k/utt2spk -o $data/dev_0.15k/segments -o $data/dev_0.15k/spk2utt -o $data/dev_0.15k/wav.scp -o $data/dev_0.15k/feats.scp -o $data/dev_0.15k/cmvn.scp \
-n dev_015k_data \
"$cmd_prefix utils/subset_data_dir.sh $data/dev_0.75kshort 150 $data/dev_0.15k"

In [ ]:
git add dvc.yaml dvc.lock

# Training and alignment

### monophone

In [ ]:
mkdir $exp/mono

In [ ]:
dvc run -f \
-d $data/train_2kshort/feats.scp -d $data/train_2kshort/utt2spk -d $data/train_2kshort/text -d $data/train_2kshort/cmvn.scp -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones.txt -d $data/lang_am_train/topo -d $data/lang_am_train/phones \
-o $exp/mono/final.mdl -o $exp/mono/tree -o $exp/mono/phones.txt \
-n mono \
"$cmd_prefix steps/train_mono.sh --boost-silence 1.25 --nj 8 $data/train_2kshort $data/lang_am_train $exp/mono"

In [ ]:
git add dvc.lock dvc.yaml

In [ ]:
dvc run -f \
-d $exp/mono/final.mdl -d $exp/mono/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/mono/graph/HCLG.fst -O $exp/mono/graph/words.txt \
-n mono_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang $exp/mono $exp/mono/graph"

In [ ]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/mono/final.mdl -d $exp/mono/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/mono.json \
-n mono_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/mono/graph $data/dev_0.15k $exp/mono/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/mono/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/mono.json"

In [ ]:
cat exp/metrics/mono.json

In [ ]:
cat exp/metrics/mono.json

In [ ]:
#base %WER 70.18
#2kshort %WER 75.96
#2kshort  + correct spk _2 %WER 75.41
#2kshort  + correct spk _2 no _I_B_S_E %WER 76.24
#2kshort  + correct spk _2 no _I_B_S_E small dev %WER 74.72 
#2kshort  + correct spk _2 no _I_B_S_E shared silence phones small dev %WER 75.25
#2kshort  + correct spk _2 no _I_B_S_E extra_questions stress %WER 75.70
######################################################################################################################
#2kshort  + 8 spk + no IBSE %WER 73.65
#2kshort  + 8 spk + no IBSE %WER extra_questions stress 74.34

### triphone

In [ ]:
# aligning train data

In [ ]:
dvc run -f \
-d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/text -d $data/train_5k/cmvn.scp -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones -d $data/lang_am_train/phones.txt -d $data/lang_am_train/oov.int -d $exp/mono/final.mdl -d $exp/mono/tree \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -O $f; done) \
-n mono_ali \
"$cmd_prefix steps/align_si.sh --boost-silence 1.25 --nj 8 $data/train_5k $data/lang_am_train $exp/mono $exp/mono_train_5k_ali"

In [ ]:
dvc run -f \
$(for f in $exp/mono_train_5k_ali/ali.*.gz; do echo -d $f; done) -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones.txt -d $data/lang_am_train/topo -d $data/lang_am_train/phones -d $data/train_5k/feats.scp -d $data/train_5k/utt2spk -d $data/train_5k/cmvn.scp \
-o $exp/tri1/final.mdl -o $exp/tri1/tree -o $exp/tri1/phones.txt \
-n tri1 \
"$cmd_prefix steps/train_deltas.sh --boost-silence 1.25 2000 10000 $data/train_5k $data/lang_am_train $exp/mono_train_5k_ali $exp/tri1; \
cp $exp/tri1/final.mdl $exp/tri1/final.mdl_copy; mv $exp/tri1/final.mdl_copy $exp/tri1/final.mdl"

In [ ]:
dvc run -f \
-d $exp/tri1/final.mdl -d $exp/tri1/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri1/graph/HCLG.fst -O $exp/tri1/graph/words.txt \
-n tri1_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang $exp/tri1 $exp/tri1/graph"

In [ ]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri1/final.mdl -d $exp/tri1/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/tri1.json \
-n tri1_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/tri1/graph $data/dev_0.15k $exp/tri1/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri1/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri1.json"

In [ ]:
cat exp/metrics/tri1.json

In [ ]:
cat exp/metrics/tri1.json

In [ ]:
# base %WER 60.95
# half + 2kshort %WER 61.70
# whole_train + 2kshort %WER 61.73
# half + 2kshort + correct spk_2 no _I_B_S_E short dev %WER 61.26
# half + 2kshort + correct spk_2 no _I_B_S_E shared silence phones %WER 63.56
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 61.51
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.72
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 56.28

### lda mllt

In [ ]:
dvc run -f \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones -d $data/lang_am_train/phones.txt -d $data/lang_am_train/oov.int -d $exp/tri1/final.mdl -d $exp/tri1/tree \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -O $f; done) \
-n tri_ali \
"$cmd_prefix steps/align_si.sh --nj 8 $data/train $data/lang_am_train $exp/tri1 $exp/tri1_train_ali"

In [ ]:
mkdir /data/exp/kostya/ukroASR/exp/tri2b

In [ ]:
dvc run -f \
$(for f in $exp/tri1_train_ali/ali.*.gz; do echo -d $f; done) -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones.txt -d $data/lang_am_train/topo -d $data/lang_am_train/phones -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri2b/final.mdl -o $exp/tri2b/final.mat -o $exp/tri2b/tree -o $exp/tri2b/phones.txt \
-n tri2b \
"$cmd_prefix steps/train_lda_mllt.sh --splice-opts \"--left-context=3 --right-context=3\" 2500 15000 $data/train $data/lang_am_train $exp/tri1_train_ali $exp/tri2b; \
cp $exp/tri2b/final.mdl $exp/tri2b/final.mdl_copy; mv $exp/tri2b/final.mdl_copy $exp/tri2b/final.mdl; \
cp $exp/tri2b/final.mat $exp/tri2b/final.mat_copy; mv $exp/tri2b/final.mat_copy $exp/tri2b/final.mat"

In [ ]:
dvc run -f \
-d $exp/tri2b/final.mdl -d $exp/tri2b/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri2b/graph/HCLG.fst -O $exp/tri2b/graph/words.txt \
-n tri2b_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang $exp/tri2b $exp/tri2b/graph"

In [ ]:
dvc run \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/graph/HCLG.fst -d $data/lang/words.txt \
-m $exp/metrics/tri2b.json \
-n tri2b_metrics \
"$cmd_prefix steps/decode.sh --nj 4 $exp/tri2b/graph $data/dev_0.15k $exp/tri2b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri2b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri2b.json"

In [ ]:
cat $exp/metrics/tri2b.json

In [ ]:
cat $exp/metrics/tri2b.json

In [ ]:
#base %WER 68.48
# half + 2kshort %WER 68.22
# full + 2kshort + correct spk %WER 67.81
# full + 2kshort + correct spk _2 %WER 65.25
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 64.77
# 2kshort + half + correct spk_2 + no _I_B_S_E shared silence phones  %WER 67.73
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 64.91
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 56.31
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 57.05

### LDA + MLLT + SAT

In [ ]:
dvc run -f \
-d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/text -d $data/train/cmvn.scp -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones -d $data/lang_am_train/phones.txt -d $data/lang_am_train/oov.int -d $exp/tri2b/final.mdl -d $exp/tri2b/final.mat -d $exp/tri2b/full.mat -d $exp/tri2b/tree \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -O $f; done) \
-n tri2b_ali \
"$cmd_prefix steps/align_si.sh --nj 8 $data/train $data/lang_am_train $exp/tri2b $exp/tri2b_train_ali"

In [ ]:
dvc run -f \
$(for f in $exp/tri2b_train_ali/ali.*.gz; do echo -d $f; done) -d $data/lang_am_train/L.fst -d $data/lang_am_train/words.txt -d $data/lang_am_train/phones.txt -d $data/lang_am_train/topo -d $data/lang_am_train/phones -d $data/train/feats.scp -d $data/train/utt2spk -d $data/train/cmvn.scp \
-o $exp/tri3b/final.mdl -o $exp/tri3b/final.mat -o $exp/tri3b/tree -o $exp/tri3b/phones.txt \
-n tri3b \
"$cmd_prefix steps/train_sat.sh 2500 15000 $data/train $data/lang_am_train $exp/tri2b_train_ali $exp/tri3b; \
cp $exp/tri3b/final.mdl $exp/tri3b/final.mdl_copy; mv $exp/tri3b/final.mdl_copy $exp/tri3b/final.mdl; \
cp $exp/tri3b/final.mat $exp/tri3b/final.mat_copy; mv $exp/tri3b/final.mat_copy $exp/tri3b/final.mat"

In [ ]:
dvc run -f \
-d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $data/lang/L.fst -d $data/lang/G.fst -d $data/lang/phones.txt -d $data/lang/words.txt -d $data/lang/phones \
-O $exp/tri3b/graph/HCLG.fst -O $exp/tri3b/graph/words.txt \
-n tri3b_graph \
"$cmd_prefix utils/mkgraph.sh $data/lang $exp/tri3b $exp/tri3b/graph"

In [ ]:
dvc run -f \
-d $data/dev_0.15k/utt2spk -d $data/dev_0.15k/feats.scp -d $data/dev_0.15k/cmvn.scp \
-d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $exp/tri3b/final.mat \
-d $exp/tri3b/graph/HCLG.fst -d $data/lang/words.txt -d $data/lang/phones \
$(for f in $exp/tri3b/decode_dev_0.15k/lat.*.gz; do echo -O $f; done) \
-m $exp/metrics/tri3b.json \
-n tri3b_metrics \
"$cmd_prefix steps/decode_fmllr.sh --nj 4 $exp/tri3b/graph $data/dev_0.15k $exp/tri3b/decode_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/decode_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri3b.json"

In [ ]:
# base %WER 54.14
# half + 2kshort %WER 53.50
# 2kshort + half + correct spk_2 + no _I_B_S_E %WER 50.55
# half + 2kshort + correct spk_2 no _I_B_S_E extra_questions stress %WER 50.85
######################################################################################################################
#2kshort + 45k + 8 spk + no IBSE %WER 47.46
#2kshort + 45k + 8 spk + no IBSE + extra_questions stress %WER 46.80

In [ ]:
dvc run -f \
$(for f in $exp/tri3b/decode_dev_0.15k/lat.*.gz; do echo -d $f; done) \
-d $data/carpa/G.carpa \
-m $exp/metrics/tri3b_rescore.json \
-n tri3b_rescore_metrics \
"$cmd_prefix steps/lmrescore_const_arpa.sh $data/lang $data/carpa $data/dev_0.15k $exp/tri3b/decode_dev_0.15k $exp/tri3b/rescore_dev_0.15k; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/rescore_dev_0.15k/scoring_kaldi/best_wer $exp/metrics/tri3b_rescore.json"

## Decode 750 utterances dev set

In [ ]:
dvc run -f \
-d $data/dev_0.75kshort/utt2spk -d $data/dev_0.75kshort/feats.scp -d $data/dev_0.75kshort/cmvn.scp \
-d $exp/tri3b/final.mdl -d $exp/tri3b/tree -d $exp/tri3b/final.mat \
-d $exp/tri3b/graph/HCLG.fst -d $data/lang/words.txt -d $data/lang/phones \
$(for i in $(seq 4); do echo -O $exp/tri3b/decode_dev_0.75kshort/lat.$i.gz; done) \
-m $exp/metrics/tri3b_0.75kshort.json \
-n tri3b_075kshort_metrics \
"$cmd_prefix steps/decode_fmllr.sh --nj 4 $exp/tri3b/graph $data/dev_0.75kshort $exp/tri3b/decode_dev_0.75kshort; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/decode_dev_0.75kshort/scoring_kaldi/best_wer $exp/metrics/tri3b_0.75kshort.json"

In [ ]:
dvc run -f \
$(for i in $(seq 4); do echo -d $exp/tri3b/decode_dev_0.75kshort/lat.$i.gz; done) \
-d $data/carpa/G.carpa \
-m $exp/metrics/tri3b_0.75kshort_rescore.json \
-n tri3b_075kshort_rescore_metrics \
"$cmd_prefix steps/lmrescore_const_arpa.sh $data/lang $data/carpa $data/dev_0.75kshort \
                                           $exp/tri3b/decode_dev_0.75kshort $exp/tri3b/rescore_dev_0.75kshort; \
$UKROASR_ROOT/parce_wer_report.sh $exp/tri3b/rescore_dev_0.175kshort/scoring_kaldi/best_wer \
                                  $exp/metrics/tri3b_0.75kshort_rescore.json"